## Model selection
### useful functions

In [4]:
import glob
import os.path
import numpy as np
import math

def print_latex_line(model_name, results_dict):
    ordered_datasets = ['SimVerb3500-dev', 'MEN-dev', 'SimLex999', 'SimLex333', 'SimVerb3500-test', 'RG65', 'SCWS','MEN-test', 'MTurk', 'WS353'] 
    print " & ".join([model_name] + ["{0:0.1f}".format(results_dict[e]*100) for e in ordered_datasets]) + "\\\\"
    
def csv_to_dict(fname, datasets, csv_sep=",", weights=[]):
    res = {}
    with open(fname, "r") as f:
        measure_names = f.readline().rstrip().split(csv_sep)
        measure_vals = f.readline().rstrip().split(csv_sep)
        measure_vals = [float(v) for v in measure_vals]
    scores = []
    for n, v in zip(measure_names, measure_vals):
        if not n or n not in datasets:
            continue
        res[n] = v
        scores.append(v)
    if sum(scores) > 0:
        if len(weights) == len(scores):
            res['mean'] = sum([res[datasets[i]] * w for i, w in enumerate(weights)])
        else:
            res['mean'] = sum(scores) / float(len(scores))
    else:
        res['mean'] = np.nan
    return res


def store_eval(embedding_root_dir, datasets, prefix = "", weights=[]):
    """
    store all results located in embedding_root_dir in a dictionary
    and adds an optional prefix at the key
    then returns the dict
    """
    results_dict = dict()
    emb_dirs = glob.glob(embedding_root_dir + "/*")
    for _dir in emb_dirs:
        eval_fname = _dir + "/eval"
        model_name = _dir.split("/")[-1]

        if not os.path.isfile(eval_fname):
            continue
        full_name = prefix+model_name
        if full_name in results_dict:
            raise ValueError()
        results_dict[prefix+model_name] = csv_to_dict(eval_fname, datasets, weights=weights)
    return results_dict

# Evaluate on full dictionary


In [6]:
emb_root = "../embeddings/en_wn_full/"
emb_root_wn_retrofitting = "../embeddings/en_wn_full_retrofitting/"

test_sets=['SimVerb3500-dev', 'MEN-dev','MEN-test', 'SimVerb3500-test', 'MEN-test', 'WS353', 'RW', 'RG65', 'SimLex999', 'MTurk', 'SCWS', 'SimLex333']
dev_sets=['SimVerb3500-dev', 'MEN-dev']
weights=[1,0.5] # weights of importance for dev_sets
embedding_eval = "../"

results_dict = {}
results_dict.update(store_eval(emb_root, dev_sets, "", weights))
results_dict.update(store_eval(emb_root_wn_retrofitting, dev_sets, "", weights))

models = results_dict.keys()
mean_scores = [results_dict[m]['mean'] for m in models]
idx_sorted = np.argsort(mean_scores)

verbose=True
best_dict = None
best_hill = None
best_ae = None
best_retrofitting = None
best_dict_notw2v = None

for i in idx_sorted:
    if verbose:
        print models[i], ":", mean_scores[i]
    if np.isnan(mean_scores[i]):
        continue
    if 'pen' in models[i]:
        best_dict = models[i]
    if 'pen' in models[i] and not 'w2v' in models[i]:
        best_dict_notw2v = models[i]
    if 'hill' in models[i]:
        best_hill = models[i]
    if 'pen0' in models[i]:
        best_ae = models[i]
    if 'retrofit' in models[i]:
        best_retrofitting = models[i]
    if verbose:
        try:
            print " ; ".join([dataset + " : " + str(results_dict[models[i]][dataset]) for dataset in dev_sets])
        except:
            print 
            
print ""
print "Full results"
if best_dict:
    print "Best CPAE w/ pretrained"
    print best_dict
    d = csv_to_dict(emb_root + "/" + best_dict + "/eval", test_sets)
    print d
    print_latex_line("CPAE", d)
if best_dict_notw2v:
    print "Best CPAE w/ pretrained not w2v"
    print best_dict_notw2v
    d = csv_to_dict(emb_root + "/" + best_dict_notw2v + "/eval", test_sets)
    print d
    print_latex_line("CPAE notw2v", d)
if best_ae:
    print "Best AE w/ pretrained"
    print best_ae
    d = csv_to_dict(emb_root + "/" + best_ae + "/eval", test_sets)
    print d
    print_latex_line("AE", d)
if best_hill:
    print "Best Hill w/ pretrained"
    print best_hill
    d = csv_to_dict(emb_root + "/" + best_hill + "/eval", test_sets)
    print d
    print_latex_line("Hill", d)
if best_hill:
    print "Best retrofitting"
    print best_retrofitting
    d = csv_to_dict(emb_root_wn_retrofitting + "/" + best_retrofitting + "/eval", test_sets)
    print d
    print_latex_line("retrofitting", d)




Full results
